In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
import os
from torchvision.transforms import v2
import torch
import shutil
from PIL import Image
from ultralytics import YOLO
import cv2

# Local dep
project_dir = '/data/konrad/workspace'
sys.path.insert(0, project_dir)

from helpers.datasets import CalfCenterFaceDataset
from helpers.helpers import get_indices, uniform_sample_with_values, load_face_data

In [2]:
def plot_bbox_gallery(image_df, n_cols=5):
    n_images = image_df.shape[0]
    n_rows = n_images // n_cols + int(n_images % n_cols > 0)
    
    plt.figure(figsize=(20, n_rows * 4))
    for i, row in image_df.iterrows():
        img = Image.open(row["path"])
        # plt.subplot(n_rows, n_cols, i + 1)
        ax = plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(img)
        # ax = plt.gca()
        score = row["conf"]
        box_x = row["box_x"]
        box_y = row["box_y"]
        box_width = row["box_width"]
        box_height = row["box_height"]
        box_x = box_x - (box_width / 2)
        box_y = box_y - (box_height / 2)
        rect = plt.Rectangle((box_x, box_y), box_width, box_height, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(box_x, box_y - 10, f' {score:.2f}', color='red', fontsize=12, backgroundcolor='white')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
def df_to_yolo(df, root_dir, img_src_dir = None):
    
    img_path = root_dir + "/" + "images"
    if not os.path.exists(img_path):
        os.makedirs(img_path)

    lbl_path = root_dir + "/" + "labels"
    if not os.path.exists(lbl_path):
        os.makedirs(lbl_path)

    for index, row in df.iterrows():
        # Extract the image name
        image_name = row['path']

        img_src_dir = row['img_dir']
        shutil.copy(os.path.join(img_src_dir, image_name), os.path.join(img_path, image_name))
        filename = os.path.splitext(image_name)[0]

        # Open the image file
        with Image.open(os.path.join(img_src_dir, image_name)) as img:
            # Get the width and height
            imgWidth, imgHeight = img.size

        class_idx = row['type']
        # class_idx = 0

        xmin = row['xmin']
        ymin = row['ymin']
        xmax = row['xmax']
        ymax = row['ymax']

        center_x = (xmin + xmax) / 2.0
        center_y = (ymin + ymax) / 2.0

        width = xmax - xmin
        height = ymax - ymin

        
        # image = cv2.imread(os.path.join(img_src_dir, image_name))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


        # plt.figure(figsize=(10, 10))
        # plt.imshow(image)
        # ax = plt.gca()
        
        # rect = plt.Rectangle((xmin, ymin), width, height, edgecolor='r', facecolor='none')
        # ax.add_patch(rect)
        # plt.axis('off')
        # plt.show()

        # print(center_x == (xmin + (width / 2)), center_x, xmin + (width / 2))
        # break
        
        norm_center_x = center_x / imgWidth
        norm_center_y = center_y / imgHeight
        
        norm_width = width / imgWidth
        norm_height = height / imgHeight
        
        box_annotation = ' '.join([str(class_idx), str(norm_center_x), str(norm_center_y), str(norm_width), str(norm_height)])+'\n'

        label_filename = lbl_path + '/' + filename + ".txt"
        anno_f = open(label_filename, 'w')
        anno_f.writelines(box_annotation)
        anno_f.close()

    print("Done !")


def delete_dir_if_exists(dir_path):
    if os.path.exists(dir_path):
        try:
            shutil.rmtree(dir_path)
            print(f"Directory '{dir_path}' deleted")
        except OSError as e:
            print(f"Error deleting directory '{dir_path}': {e}")

In [3]:
ROOT_DIR = "/data/konrad/workspace"
DATA_FILE = ROOT_DIR + '/datasets/CompleteDatasetNormalFace/Face_annotations.csv'
IMAGE_DIR = ROOT_DIR + "/datasets/CompleteDatasetNormalFace"

face_df, labels, label2id, id2label = load_face_data(DATA_FILE, IMAGE_DIR)
face_df["img_dir"] = IMAGE_DIR
face_df["type"] = 0

# DATA_FILE = '/data/konrad/datasets/CompleteDatasetNormalProfilDroit/Profil_droit_annotations.csv'
# IMAGE_DIR = "/data/konrad/datasets/CompleteDatasetNormalProfilDroit"

# rigth_df, _, _, _ = load_face_data(DATA_FILE, IMAGE_DIR, extract_name=False)
# rigth_df["img_dir"] = IMAGE_DIR
# rigth_df["type"] = 1


# DATA_FILE = '/data/konrad/datasets/CompleteDatasetNormalProfilGauche/Profil_gauche_annotations.csv'
# IMAGE_DIR = "/data/konrad/datasets/CompleteDatasetNormalProfilGauche"

# left_df, _, _, _ = load_face_data(DATA_FILE, IMAGE_DIR, extract_name=False)
# left_df["img_dir"] = IMAGE_DIR
# left_df["type"] = 2

mix_df = pd.concat([face_df], ignore_index=True)
# mix_df = pd.concat([face_df, rigth_df, left_df], ignore_index=True)
# mix_df['type'] = 0 Only for any detector

filter_values = {
    'calf': ['6842', '6436', '6864'],  # Specify the values you want to include
}

valid_df = mix_df[mix_df['calf'].isin(filter_values['calf'])]
train_df = mix_df[~ mix_df['calf'].isin(filter_values['calf'])]

IMAGE_DIR = ROOT_DIR + "/datasets/calfs_face"
delete_dir_if_exists(IMAGE_DIR)
df_to_yolo(train_df, IMAGE_DIR + '/train', IMAGE_DIR)
df_to_yolo(valid_df, IMAGE_DIR + '/test', IMAGE_DIR)

Directory '/data/konrad/workspace/datasets/calfs_face' deleted
Done !
Done !


In [4]:
# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

results = model.train(data="yolo-face.yml", epochs=10, project=ROOT_DIR + "/training_log", name="yolo_face", cfg="config.yml")

model.export()

New https://pypi.org/project/ultralytics/8.2.64 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=yolo-face.yml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/konrad/workspace/training_log, name=yolo_face, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=F

train: Scanning /data/konrad/workspace/datasets/calfs_face/train/labels... 178 images, 0 backgrounds, 0 corrupt: 100%|██████████| 178/178 [00:00<00:00, 2956.14it/s]

train: New cache created: /data/konrad/workspace/datasets/calfs_face/train/labels.cache



val: Scanning /data/konrad/workspace/datasets/calfs_face/test/labels... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<00:00, 2797.03it/s]

val: New cache created: /data/konrad/workspace/datasets/calfs_face/test/labels.cache


...

                   all          9          9      0.747      0.658      0.837      0.516



10 epochs completed in 0.004 hours.
Optimizer stripped from /data/konrad/workspace/training_log/yolo_face/weights/last.pt, 6.2MB
Optimizer stripped from /data/konrad/workspace/training_log/yolo_face/weights/best.pt, 6.2MB

Validating /data/konrad/workspace/training_log/yolo_face/weights/best.pt...
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 31.67it/s]


                   all          9          9      0.968      0.667      0.797      0.568
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /data/konrad/workspace/training_log/yolo_face
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (AMD Ryzen Threadripper PRO 5975WX 32-Cores)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/data/konrad/workspace/training_log/yolo_face/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 0.8s, saved as '/data/konrad/workspace/training_log/yolo_face/weights/best.torchscript' (11.9 MB)

Export complete (2.1s)
Results saved to /data/konrad/workspace/training_log/yolo_face/weights
Predict:         yolo predict task=detect model=/data/konrad/workspace/training_log/yolo_face/weights/best.torchscript imgsz=640 

'/data/konrad/workspace/training_log/yolo_face/weights/best.torchscript'